In [ ]:
# | default_exp routes.instance_config


In [ ]:
# | export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de


In [ ]:
# | export
async def get_allowlist(
    auth: dmda.DomoFullAuth,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False,
) -> rgd.ResponseGetData:

    if auth.__class__.__name__ != "DomoFullAuth":
        raise dmda.InvalidAuthTypeError(
            function_name="get_allowlist",
            domo_instance=auth.domo_instance,
            required_auth_type=dmda.DomoFullAuth,
        )

    url = f"https://{auth.domo_instance}.domo.com/admin/companysettings/whitelist"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        headers={"accept": "*/*"},
        session=session,
        debug_api=debug_api,
        is_follow_redirects=True,
        return_raw=return_raw,
    )

    return res

#### sample implementation of get_allowlist with the wrong auth type


In [ ]:
import os
import requests

auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    await get_allowlist(auth=auth)
except dmda.InvalidAuthTypeError as e:
    print(e)

get_allowlist: This API rquires DomoFullAuth at domo-dojo


#### sample implementation of get_allowlist with the correct auth type


In [ ]:
import os
import requests

auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

res = await get_allowlist(auth=auth)

res

ResponseGetData(status=200, response={'addresses': ['0.0.0.0/0', '20.171.67.126']}, is_success=True)

In [ ]:
# | export
class Allowlist_UnableToUpdate(de.DomoError):
    def __init__(
        self, 
        status: int, reason: str, domo_instance:str, function_name: str = "update_allowlist"
    ):

        super().__init__(
            function_name=function_name,
            status=status,
            message=f"unable to update allowlist: {reason}",
            domo_instance=domo_instance,
        )

In [ ]:
# | export


async def set_allowlist(
    auth: dmda.DomoAuth,
    ip_address_ls: list[str],
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None
) -> rgd.ResponseGetData:
    """companysettings/whitelist API only allows users to SET the allowlist does not allow INSERT or UPDATE"""

    url = f"https://{auth.domo_instance}.domo.com/admin/companysettings/whitelist"

    body = {"addresses": ip_address_ls}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        is_follow_redirects=True,
        return_raw=return_raw,
        session = session,
        headers={"accept": "text/plain"},
    )
    if not res.is_success:
        raise Allowlist_UnableToUpdate(
            status=res.status, reason=res.response, domo_instance=auth.domo_instance
        )

    if res.is_success:
        res.response = f"allow list updated from {res.response}"

    return res

In [ ]:
import os

ip_address_ls = ["0.0.0.0/0"]

auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

auth = dmda.DomoFullAuth(
    domo_instance="domo-dojo",
    domo_username="jae@onyxreporting.com",
    domo_password=os.environ["DOJO_PASSWORD"],
)

res = await set_allowlist(auth=auth, ip_address_ls=ip_address_ls)
res

ResponseGetData(status=200, response='allow list updated from 20.171.67.126', is_success=True)

In [ ]:
# # | export
# async def update_authorized_domains(
#     auth: DomoAuth,
#     authorized_domain_list: list[str],
#     debug_api: bool = False,
#     session: httpx.AsyncClient = None,
# ):
#     url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/authorized-domains"

#     body = {"name": "authorized-domains", "value": ",".join(authorized_domain_list)}

#     if debug_api:
#         print(url)

#     res = await gd.get_data(
#         auth=auth,
#         url=url,
#         method="PUT",
#         body=body,
#         log_results=log_results,
#         debug_api=debug_api,
#         session=session,
#     )

#     return res


In [ ]:
# # | export
# async def get_authorized_domains(
#     auth: DomoAuth,
#     debug_api: bool = False,
#     log_results: bool = False,
#     session: httpx.AsyncClient = None,
# ):
#     url = f"https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/authorized-domains"

#     if debug_api:
#         print(url)

#     res = await gd.get_data(
#         auth=auth,
#         url=url,
#         method="GET",
#         log_results=log_results,
#         debug_api=debug_api,
#         session=session,
#     )

#     return res


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()


Bad pipe message: %s [b'}p\xce\xd0\x0f\x91T\xbe\x98\xc9\x8aZp\xd9m\x06\xf6\x9e \xb7\xe8\x97\xbd\xc5\xc2\xbe\x17Yx,e', b'9\x81\x7f\x8c\xe5`\xce\xc2`\x98J\x1dV\x9f\x1bLX\xf0\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00']
Bad pipe message: %s [b'\xda\xc2\xe7gB\xb0\x08\xd4H\x08~\x0e\x08\xa4\x84\xcb\xb9s\x00\x00>\xc0\x14\xc0\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t\x003\x002\x001\x000\xc0\x0e\xc0\x04\x00/\x00\x9a\x00\x99\x00\x98\x00\x97\x00\x96\x00\x07\xc0\x11\xc0\x07\xc0\x0c\xc0\x02\x00\x05\x00\x04\x00\xff\x02\x01\x00\x00C\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x1c\x00\x1a\x00\x17\x00\x19\x00\x1c\x00\x1b\x00\x18\x00\x1a\x00\x16\x00\x0e\x00\r\x00\x0b\x00', b'\t\x00\n\x00#\x00\x00\x00\x0f\x00\x01']
Bad pipe message: %s [b'', b'\x00\x02']
Bad pipe message: %s [b'\x8e\x1e\x8a\x1fV\x97\xc4\x9a\x93\x00\x1bS\xa40\xe2\xe8u\xc2\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x8